In [1]:
import pandas as pd
import numpy as np
from catnip.fla_redshift import FLA_Redshift
from sqlalchemy import null
from datetime import datetime

from prefect.blocks.system import Secret
from typing import Dict
from concurrent.futures import ThreadPoolExecutor

In [2]:
def get_redshift_credentials() -> Dict:

    cred_dict = {
        "dbname": Secret.load("stellar-redshift-db-name").get(),
        "host": Secret.load("stellar-redshift-host").get(),
        "port": 5439,
        "user": Secret.load("stellar-redshift-user-name").get(),
        "password": Secret.load("stellar-redshift-password").get(),

        "aws_access_key_id": Secret.load("fla-s3-aws-access-key-id-east-1").get(),
        "aws_secret_access_key": Secret.load("fla-s3-aws-secret-access-key-east-1").get(),
        "bucket": Secret.load("fla-s3-bucket-name-east-1").get(),
        "subdirectory": "us-east-1",

        "verbose": False,
    }

    return cred_dict

with ThreadPoolExecutor(1) as pool:
    rs_creds = pool.submit(lambda: get_redshift_credentials()).result()

In [3]:
q = """
SELECT
    *
FROM
    custom.historical_tv_ratings
"""

df = FLA_Redshift(**rs_creds).query_warehouse(sql_string = q)

In [ ]:
mia_ftl = df[['season','event_date','hh_viewership_mia_ftl_game','hh_ratings_mia_ftl_game'
              ]].rename(columns = {'hh_viewership_mia_ftl_game' : 'viewership', 'hh_ratings_mia_ftl_game' : 'ratings'})

mia_ftl['county_1'] = 'Broward'
mia_ftl['county_2'] = 'Miami-Dade'
mia_ftl['county_3'] = 'Monroe'
mia_ftl['county_4'] = np.nan
mia_ftl['county_5'] = np.nan

pb = df[['season','event_date','hh_viewership_pb_game','hh_ratings_pb_game'
         ]].rename(columns = {'hh_viewership_pb_game' : 'viewership', 'hh_ratings_pb_game' : 'ratings'})

pb['county_1'] = 'Palm'
pb['county_2'] = 'Miami-Dade'
pb['county_3'] = 'Monroe'
pb['county_4'] = np.nan
pb['county_5'] = np.nan

fm = df[['season','event_date','hh_viewership_fm_game','hh_ratings_fm_game'
         ]].rename(columns = {'hh_viewership_fm_game' : 'viewership', 'hh_ratings_fm_game' : 'ratings'})

fm['county_1'] = 'Broward'
fm['county_2'] = 'Miami-Dade'
fm['county_3'] = 'Monroe'
fm['county_4'] = np.nan
fm['county_5'] = np.nan

final = pd.concat(objs = [mia_ftl,pb,fm], axis = 0)

final

,season,event_date,viewership,ratings,county_1,county_2,county_3,county_4,county_5,loc
0,2022-23,2022-10-13,NaN,0.58,Broward,Miami-Dade,Monroe,NaN,NaN,NaN
1,2022-23,2022-10-15,NaN,0.51,Broward,Miami-Dade,Monroe,NaN,NaN,NaN
2,2022-23,2022-10-17,NaN,0.57,Broward,Miami-Dade,Monroe,NaN,NaN,NaN
3,2022-23,2022-10-19,NaN,NaN,Broward,Miami-Dade,Monroe,NaN,NaN,NaN
4,2022-23,2022-10-21,NaN,NaN,Broward,Miami-Dade,Monroe,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
335,2025-26,2025-12-29,2743.0,0.53,NaN,NaN,NaN,NaN,NaN,fm
336,2025-26,2025-12-30,3409.0,0.66,NaN,NaN,NaN,NaN,NaN,fm
337,2025-26,2026-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fm
338,2025-26,2026-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fm


In [ ]:
# historical_tv_ratings_map = pd.read_csv("C:\\Users\\riffere\\Desktop\\historical_tv_ratings_map.csv")
# FLA_Redshift(**rs_creds).write_to_warehouse(df = historical_tv_ratings_map, table_name= "historical_tv_ratings_map")

In [3]:
historical_tv_ratings = pd.read_csv("C:\\Users\\riffere\\Florida Panthers\\SP-BS - Documents\\Data Science\\Projects\\Manual-Loads\\historical_tv_ratings.csv")

In [6]:
def calc_rolling_avg(df: pd.DataFrame) -> pd.DataFrame:

    # make sure event_dates are date fields and sorted properly

    df['event_date'] = pd.to_datetime(df['event_date'])
    df = df.sort_values(by =['event_date'], ascending = True).reset_index(drop= True)

    # calculate overall avg by game and then rolling avg by season
    
    subset = df[['hh_ratings_mia_ftl','hh_ratings_pb','hh_ratings_fm']]
    avg_rows = subset.mean(axis = 1)
    df['overall_avg_rating'] = avg_rows
    df['running_avg_rating'] = df.groupby('season')['overall_avg_rating'].expanding().mean().reset_index(level=0, drop=True)
    df.loc[df['overall_avg_rating'].isnull(), 'running_avg_rating'] = np.nan

    return df

In [7]:
calc_rolling_avg(historical_tv_ratings)

,season,event_date,game_number,channel,hh_viewership_mia_ftl,hh_ratings_mia_ftl,hh_viewership_pb,hh_ratings_pb,hh_viewership_fm,hh_ratings_fm,unique_streams,total_viewership,start_time,team,loc,overall_avg_rating,running_avg_rating
0,2022-23,2022-10-13,1,BSN Florida,NaN,0.58,NaN,NaN,NaN,NaN,NaN,NaN,7:30 PM,New York Islanders,Away,0.58,0.580000
1,2022-23,2022-10-15,2,BSN Florida,NaN,0.51,NaN,NaN,NaN,NaN,NaN,NaN,1:00 PM,Buffalo Sabres,Away,0.51,0.545000
2,2022-23,2022-10-17,3,BSN Florida,NaN,0.57,NaN,NaN,NaN,NaN,NaN,NaN,7:00 PM,Boston Bruins,Away,0.57,0.553333
3,2022-23,2022-10-19,4,TNT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7:30 PM,Philadelphia Flyers,Home,NaN,NaN
4,2022-23,2022-10-21,5,"Hulu, ESPN+",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7:30 PM,Tampa Bay Lightning,Home,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
335,2025-26,2026-04-07,78,Scripps,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7:00 PM,Montreal Canadiens,Away,NaN,NaN
336,2025-26,2026-04-09,79,Scripps,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7:00 PM,Ottawa Senators,Away,NaN,NaN
337,2025-26,2026-04-11,80,Scripps,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7:00 PM,Toronto Maple Leafs,Away,NaN,NaN
338,2025-26,2026-04-13,81,Scripps,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7:00 PM,New York Rangers,Home,NaN,NaN
